# lab-10-6-1_Advanced-CNN(ResNet)

https://github.com/pytorch/vision/blob/814c4f08bc9296a314ecb9e216648e0d059a3edc/torchvision/models/resnet.py#L252

In [1]:
import torch
from torch import Tensor
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

In [2]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}

## conv3x3 and conv1x1

In [3]:
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

## BasicBlock

In [4]:
class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int, # 채널 수
        planes: int, # 채널 수
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        # x.shape = 3x64x64
        identity = x
        # identity.shape = 3x64x64

        out = self.conv1(x) # 3x3 stride=stride
        # out.shape = 3x64x64(stride=1) or 3x32x32(stride=2)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride=1
        # out.shape = 3x64x64(stride=1) or 3x32x32(stride=2)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

## Bottleneck

In [5]:
class Bottleneck(nn.Module):
    expansion: int = 4

    def __init__(
        self,
        inplanes: int, # 채널 수
        planes: int, # 채널 수
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width) # conv1x1(512, 128)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation) # conv3x3(128, 128)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion) # conv1x1(128, 512)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        # x.shape = 3x64x64
        identity = x
        # identity.shape = 3x64x64

        out = self.conv1(x) # 1x1 stride=1
        # out.shape = 3x64x64
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride=stride
        # out.shape = 3x64x64(stride=1) or 3x32x32(stride=2)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out) # 1x1 stride=1
        # out.shape = 3x64x64(stride=1) or 3x32x32(stride=2)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

## ResNet

In [ ]:
# resnet50 = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)

class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:

        super(ResNet, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1

        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group

        # input.shape = 3x224x224
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        # 64x112x112

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # 64x56x56

        self.layer1 = self._make_layer(block, 64, layers[0]'''3''')
        self.layer2 = self._make_layer(block, 128, layers[1]'''4''', stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2]'''6''', stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3]'''3''', stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    # self.inplanes = 64
    # self.dilation = 1
    # self.layer1 = self._make_layer(Bottleneck, 64, 3)
    # self.layer2 = self._make_layer(Bottleneck, 128, 4, stride=2, dilate=False)
    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride), # conv1x1(256, 512, 2)
                norm_layer(planes * block.expansion), # norm_layer(512)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer))
        # layers.append(Bottleneck(256, 128, 2, downsample, 1, 64, 1, norm_layer))
        
        self.inplanes = planes * block.expansion
        # self.inplanes = 512
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=norm_layer))
            # layers.append(Bottleneck(512, 128, 1, 64, 1, norm_layer))
            # layers.append(Bottleneck(512, 128, 1, 64, 1, norm_layer))
            # layers.append(Bottleneck(512, 128, 1, 64, 1, norm_layer))
            # layers.append(Bottleneck(512, 128, 1, 64, 1, norm_layer))

        return nn.Sequential(*layers)

        '''self.layer1 = nn.Sequential(
            Bottleneck(64, 64, 1, downsample, 1, 64, 1, norm_layer)
            Bottleneck(256, 64, 1, 64, 1, norm_layer)
            Bottleneck(256, 64, 1, 64, 1, norm_layer)
            Bottleneck(256, 64, 1, 64, 1, norm_layer)
        )
        '''
        '''self.layer2 = nn.Sequential(
            Bottleneck(256, 128, 2, downsample, 1, 64, 1, norm_layer)
            Bottleneck(512, 128, 1, 64, 1, norm_layer)
            Bottleneck(512, 128, 1, 64, 1, norm_layer)
            Bottleneck(512, 128, 1, 64, 1, norm_layer)
            Bottleneck(512, 128, 1, 64, 1, norm_layer)
        )
        '''

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

        # self.layer1 = self._make_layer(Bottleneck, 64, 3)

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [ ]:
def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model

In [ ]:
def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model

In [ ]:
def resnet152(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model

In [6]:
import torchvision.models.resnet as resnet

In [7]:
resnet50 = resnet.resnet50()

In [8]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 